In [83]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from time import time
from ultralytics import YOLO
from keras.preprocessing import image
from keras.models import Model
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
#from dgl.nn.pytorch.conv import RelGraphConv
from dgl.nn import GraphConv
#import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import cv2
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input

global_resize_image=(256,256)

In [84]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [85]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


In [86]:
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
#filename = 'Flickr_1k.trainImages.txt'
filename='Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 6000


In [87]:
images = 'Flicker8k_Dataset/'

# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [88]:
print(images)

Flicker8k_Dataset/


In [89]:
train_images_file = 'Flickr_8k.trainImages.txt'
# Read the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images

In [90]:
# Below file conatains the names of images to be used in test data
test_images_file = 'Flickr_8k.testImages.txt'
# Read the validation image names in a set# Read the test image names in a set
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

# Create a list of all the test images with their full path names
test_img = []

for i in img: # img is list of full path names of all images
    if i[len(images):] in test_images: # Check if the image belongs to test set
        test_img.append(i) # Add it to the list of test images

In [91]:
len(test_img)

1000

In [92]:
def preprocess(image_path):
    
    img = image.load_img(image_path, target_size=(224, 224))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [93]:
# Load pre-trained VGG19 model
vgg19 = models.vgg19(pretrained=True)
vgg19.classifier = vgg19.classifier[:-1]
vgg19.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [94]:
# Convert to DataFrame
def Convert_to_DataFrame(results):
    boxes = results[0].boxes.xyxy.tolist()
    classes = results[0].boxes.cls.tolist()
    names = results[0].names
    confidences = results[0].boxes.conf.tolist()
    data = {'xmin': [box[0] for box in boxes],
        'ymin': [box[1] for box in boxes],
        'xmax': [box[2] for box in boxes],
        'ymax': [box[3] for box in boxes],
        'confidence': confidences,
        'class': classes,
        'name': [names[int(cls)] for cls in classes]}
    output_df = pd.DataFrame(data)
    #print(output_df)
    return output_df

In [95]:
#Object Detection using YOLOv8
def detect_objects(image):
    #image=cv2.imread(imagePath)
    image=cv2.resize(image,global_resize_image)
    model = YOLO('yolov8x.pt')
    results=model(image)
    results=Convert_to_DataFrame(results)
    return results 

In [96]:
#results=detect_objects(imagePath)
#print(len(results[0].boxes))
#print(results)

In [97]:
# Visualize the results
'''for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    #im_array = im_bgr.numpy()
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    #r.show()
    plt.imshow(numpy.array(im_rgb))
    plt.show()

    # Save results to disk
    #r.save(filename=f'results{i}.jpg')'''

"for i, r in enumerate(results):\n    # Plot results image\n    im_bgr = r.plot()  # BGR-order numpy array\n    #im_array = im_bgr.numpy()\n    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image\n\n    # Show results to screen (in supported environments)\n    #r.show()\n    plt.imshow(numpy.array(im_rgb))\n    plt.show()\n\n    # Save results to disk\n    #r.save(filename=f'results{i}.jpg')"

In [98]:
"""#<==================  Region Proposal ======================>


def roi_propose(boxes):
    # Function to calculate IoU
    def calculate_iou(box1, box2):
        # Calculate intersection rectangle coordinates
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        # Calculate area of intersection rectangle
        intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)
        
        # Calculate area of both bounding boxes
        box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
        box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
        
        # Calculate IoU
        iou = intersection_area / float(box1_area + box2_area - intersection_area)
        
        return iou

    # Calculate IoU matrix
    num_boxes = len(boxes)
    iou_matrix = np.zeros((num_boxes, num_boxes))
    for i in range(num_boxes):
        for j in range(i + 1, num_boxes):
            iou_matrix[i, j] = calculate_iou(boxes[i], boxes[j])
            iou_matrix[j, i] = iou_matrix[i, j]  # IoU is symmetric

    # Define threshold for IoU
    iou_threshold = 0.5

    # Merge bounding boxes based on IoU
    merged = set()
    groups = []
    for i in range(num_boxes):
        if i not in merged:
            group = [i]
            for j in range(i + 1, num_boxes):
                if j not in merged and iou_matrix[i, j] >= iou_threshold:
                    group.append(j)
                    merged.add(j)
            groups.append(group)
    # Merge regions completely inside others
    for i, box1 in enumerate(boxes):
        for j, box2 in enumerate(boxes):
            if i != j and calculate_iou(box1, box2) == 1:
                if i in merged:
                    groups.remove(groups[i])
                elif j in merged:
                    groups.remove(groups[j])
    # Create ROIs based on merged bounding boxes
    rois = {}
    for idx, group in enumerate(groups):
        roi_boxes = [boxes[i] for i in group]
        min_x = min(box[0] for box in roi_boxes)
        min_y = min(box[1] for box in roi_boxes)
        max_x = max(box[2] for box in roi_boxes)
        max_y = max(box[3] for box in roi_boxes)
        rois[idx] = (min_x, min_y, max_x, max_y)

    # Function to check if two ROIs overlap
    def is_inside(roi1, roi2):
        x1_min, y1_min, x1_max, y1_max = roi1
        x2_min, y2_min, x2_max, y2_max = roi2
        return (x1_min >= x2_min and y1_min >= y2_min and
                x1_max <= x2_max and y1_max <= y2_max)

    # Function to merge ROIs if one ROI completely encompasses the other
    def merge_overlapping_rois(rois):
        merged_rois = {}
        for idx, roi in rois.items():
            merge = True
            for merged_idx, merged_roi in merged_rois.items():
                if is_inside(roi, merged_roi):
                    # ROI is completely inside another ROI, don't merge
                    merge = False
                    break
                elif is_inside(merged_roi, roi):
                    # Another ROI is completely inside this ROI, merge
                    merged_rois[merged_idx] = roi
                    merge = False
                    break
            if merge:
                merged_rois[idx] = roi
        return merged_rois
    # Merge overlapping ROIs
    merged_rois = merge_overlapping_rois(rois)

    # Print merged ROIs
    for idx, roi in enumerate(merged_rois):
        print(f"ROI {idx}: {roi}")
    '''    
    # Print proposed ROIs
    for roi_id, roi_coords in rois.items():
        print(f"ROI {roi_id}: {roi_coords}")'''
    return merged_rois    
"""

'#<==================  Region Proposal ======================>\n\n\ndef roi_propose(boxes):\n    # Function to calculate IoU\n    def calculate_iou(box1, box2):\n        # Calculate intersection rectangle coordinates\n        x1 = max(box1[0], box2[0])\n        y1 = max(box1[1], box2[1])\n        x2 = min(box1[2], box2[2])\n        y2 = min(box1[3], box2[3])\n        \n        # Calculate area of intersection rectangle\n        intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)\n        \n        # Calculate area of both bounding boxes\n        box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)\n        box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)\n        \n        # Calculate IoU\n        iou = intersection_area / float(box1_area + box2_area - intersection_area)\n        \n        return iou\n\n    # Calculate IoU matrix\n    num_boxes = len(boxes)\n    iou_matrix = np.zeros((num_boxes, num_boxes))\n    for i in range(num_boxes):\n        for j 

In [99]:
def extract_nodes(image):
    output_df=detect_objects(image)
    #proposed_roi = roi_propose(results[0].boxes.xyxy.tolist()) #Roi calculation
    #output_df=Convert_to_DataFrame(results)
    image_nodes=[]
    for i in range(output_df.shape[0]):
        #print(output_df.iloc[i]['name'])
        data={"object_id":i,
        "start_point":(round(output_df.iloc[i]['xmin']),round(output_df.iloc[i]['ymin'])),
        "ending_point":(round(output_df.iloc[i]['xmax']),round(output_df.iloc[i]['ymax'])),
        "label":output_df.iloc[i]['name']
        }
        image_nodes.append(data)
    return image_nodes    

In [100]:
#<====================  Visualize and save the roi drawn images ===============>

'''import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Function to visualize ROIs on the image
def visualize_rois(image_path, rois, save_path):
    # Read the image
    image=cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,global_resize_image)

    # Create figure and axes
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    # Draw ROIs on the image
    for roi_id, roi_coords in rois.items():
        min_x, min_y, max_x, max_y = roi_coords
        width = max_x - min_x
        height = max_y - min_y
        rect = patches.Rectangle((min_x, min_y), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        # Add text label
        ax.text(min_x, min_y - 5, f'ROI {roi_id}', color='r')

    # Remove axis ticks
    ax.axis('off')

    # Save the image
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)

    # Show the image
    plt.show()

# Define image path and save path
#image_path = 'your_image.jpg'
save_path = 'image_with_rois.jpg'

# Visualize ROIs on the image and save
visualize_rois(imagePath, roi_propose(results[0].boxes.xyxy.tolist()), save_path)'''


"import matplotlib.pyplot as plt\nimport matplotlib.patches as patches\n\n# Function to visualize ROIs on the image\ndef visualize_rois(image_path, rois, save_path):\n    # Read the image\n    image=cv2.imread(imagePath)\n    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)\n    image=cv2.resize(image,global_resize_image)\n\n    # Create figure and axes\n    fig, ax = plt.subplots()\n\n    # Display the image\n    ax.imshow(image)\n\n    # Draw ROIs on the image\n    for roi_id, roi_coords in rois.items():\n        min_x, min_y, max_x, max_y = roi_coords\n        width = max_x - min_x\n        height = max_y - min_y\n        rect = patches.Rectangle((min_x, min_y), width, height, linewidth=2, edgecolor='r', facecolor='none')\n        ax.add_patch(rect)\n\n        # Add text label\n        ax.text(min_x, min_y - 5, f'ROI {roi_id}', color='r')\n\n    # Remove axis ticks\n    ax.axis('off')\n\n    # Save the image\n    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)\n\n    # Show 

In [101]:
def generate_nodeImages(image,nodes):
    #image=cv2.imread(imagePath)
    image=cv2.resize(image,(256,256))
    #image = cv2.rectangle(image, (106,19), (188,133), (255, 0, 0) , 2) 
    #crop=image[19:133,106:188]
    #cv2.imshow('image',crop)
    #cv2.waitKey()
    segment_array=[]
    for i in range(len(nodes)):
        start=nodes[i].get('start_point')
        end=nodes[i].get('ending_point')
        #print(end[1])
        a,b,c,d=start[1],end[1],start[0],end[0]
        crop=image[a:b,c:d]
        segment_array.append(crop)
    segment_array.append(image)    
    return segment_array    

In [102]:
# Function to extract features from an image using VGG19

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features(image):
    # Preprocess the image
    img_tensor = preprocess(image).unsqueeze(0)
    img_tensor = img_tensor.to(device)
    # Forward pass through VGG19
    with torch.no_grad():
        features = vgg19(img_tensor)
    features_numpy = features.cpu().detach().numpy()
    return features_numpy.flatten()

def featuresVGG(segment_array):
    VGG_features=[]
    for image in segment_array:
        # Convert the NumPy array to a PIL image
        image = Image.fromarray(np.uint8(image))
        
        # Extract features using VGG19
        features = extract_features(image)
        VGG_features.append(features.reshape(1, -1))
    return VGG_features

In [103]:
image=cv2.imread(r"Flicker8k_Dataset\\109202801_c6381eef15.jpg")

In [104]:
nodes=extract_nodes(image)


0: 640x640 1 person, 2 horses, 12.0ms
Speed: 0.0ms preprocess, 12.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


In [105]:
len(nodes)

3

In [106]:
segment_array=generate_nodeImages(image,nodes)

In [107]:
VGG=featuresVGG(segment_array)

In [108]:
len(segment_array)

4

In [109]:
VGG[2].shape

(1, 4096)

In [110]:
"""def graph_features(feature_list):
    
    g = dgl.DGLGraph()
    num_nodes = len(feature_list)
    g.add_nodes(num_nodes)
    features = torch.tensor(feature_list, dtype=torch.float32)
    g.ndata['features'] = features
    num_edges = num_nodes*num_nodes-1 
    g.add_edges(torch.randint(num_nodes, (num_edges,)), torch.randint(num_nodes, (num_edges,)))
    g.edata['edge_weights'] = nn.Parameter(torch.rand(num_edges, requires_grad=True))

# Define a Graph Convolutional Network (GCN) model
    class GCN(nn.Module):
        def __init__(self, in_feats, hidden_feats):
            super(GCN, self).__init__()
            self.conv = GraphConv(in_feats, hidden_feats)

        def forward(self, g, features):
            # Perform graph convolution with learnable edge weights
            h = self.conv(g, features)
            h = F.relu(h)
            return h

# Reshape the features tensor
    features = features.squeeze(1)

    # Instantiate the GCN model
    in_feats = features.shape[1] # Number of input features
    hidden_feats = 256 # Number of hidden units
    model = GCN(in_feats, hidden_feats)

    # Forward pass
    output_gcn = model(g, features)
    output=output_gcn.detach().numpy()
    output=output.transpose()
    #b=output.reshape(1,-1)
    return output"""

"def graph_features(feature_list):\n    \n    g = dgl.DGLGraph()\n    num_nodes = len(feature_list)\n    g.add_nodes(num_nodes)\n    features = torch.tensor(feature_list, dtype=torch.float32)\n    g.ndata['features'] = features\n    num_edges = num_nodes*num_nodes-1 \n    g.add_edges(torch.randint(num_nodes, (num_edges,)), torch.randint(num_nodes, (num_edges,)))\n    g.edata['edge_weights'] = nn.Parameter(torch.rand(num_edges, requires_grad=True))\n\n# Define a Graph Convolutional Network (GCN) model\n    class GCN(nn.Module):\n        def __init__(self, in_feats, hidden_feats):\n            super(GCN, self).__init__()\n            self.conv = GraphConv(in_feats, hidden_feats)\n\n        def forward(self, g, features):\n            # Perform graph convolution with learnable edge weights\n            h = self.conv(g, features)\n            h = F.relu(h)\n            return h\n\n# Reshape the features tensor\n    features = features.squeeze(1)\n\n    # Instantiate the GCN model\n    in_f

In [111]:
x=np.array(VGG)

In [112]:
x.shape

(4, 1, 4096)

In [113]:
x=x.reshape(4096,len(VGG))

In [114]:
x.shape

(4096, 4)

In [115]:
def feature_reduction(features):
    pca = PCA(n_components=1)
    pca.fit(features)
    s=StandardScaler()
    x=pca.transform(features)
    x=np.reshape(x,x.shape[0])
    return x

In [116]:
y=feature_reduction(x)

In [117]:
y.shape

(4096,)

In [118]:
def complete_extraction(image_path):
    img= cv2.imread(image_path)
    nodes=extract_nodes(img)
    segment_array=generate_nodeImages(img,nodes)
    feature_vectors=featuresVGG(segment_array)
    x=np.array(feature_vectors)
    x=x.reshape(4096,len(feature_vectors))
    feature=feature_reduction(x)
    return feature

In [119]:
m=complete_extraction("Flicker8k_Dataset\\109202801_c6381eef15.jpg")


0: 640x640 1 person, 2 horses, 12.7ms
Speed: 0.0ms preprocess, 12.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


In [120]:
m.shape

(4096,)

In [121]:
start = time()
encoding_train = {}
#train_data=[]
i=0
for img in train_img:
    
    encoding_train[img[len(images):]] = complete_extraction(img)

print("Time taken in seconds =", time()-start)


0: 640x640 1 person, 1 bench, 15.5ms
Speed: 0.0ms preprocess, 15.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 dogs, 23.4ms
Speed: 0.0ms preprocess, 23.4ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 1 kite, 15.8ms
Speed: 0.0ms preprocess, 15.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 bench, 1 dog, 7.0ms
Speed: 0.0ms preprocess, 7.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 12.0ms
Speed: 0.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 12.5ms
Speed: 0.0ms preprocess, 12.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 dog, 19.1ms
Speed: 2.8ms preprocess, 19.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 dog, 1 sports ball, 26.1ms
Speed: 0.0ms preprocess, 26.1ms inference, 0.0ms 

In [122]:
import pickle
with open("encoded_regionTrian.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)

In [123]:
start = time()
encoding_test = {}
for img in test_img:
    encoding_test[img[len(images):]] = complete_extraction(img)
print("Time taken in seconds =", time()-start)


0: 640x640 2 persons, 2 cars, 45.8ms
Speed: 2.0ms preprocess, 45.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 10.1ms
Speed: 0.0ms preprocess, 10.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1 bench, 1 dog, 18.8ms
Speed: 0.0ms preprocess, 18.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 dog, 7.9ms
Speed: 0.0ms preprocess, 7.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 backpack, 1 sports ball, 1 hair drier, 4.5ms
Speed: 6.0ms preprocess, 4.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 birds, 1 dog, 11.5ms
Speed: 0.7ms preprocess, 11.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 dog, 1 sheep, 20.9ms
Speed: 0.0ms preprocess, 20.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 19.6ms
Speed: 0.0ms pr

In [124]:
with open("encoded_regionTest.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_test, encoded_pickle)